# Accelerate Your Development With FsHttp and FSharp.Data

So you have to hit a new JSON API. Is there an SDK library? Nope.

Forgot what flag you need for cURL? Time to use Postman. Postman asking you to sign in? Hmm... maybe you want a notebook experience? `pip install requests` but your Python installation is broken again....

You think to yourself,

> _there has to be a better way!_

Well, fortunately, there is. And today I'm going to help you set it up in a matter of minutes.

## Installation 🧑‍💻
### _.NET runs on Mac?!_

First, you need to install [dot.net](https://dot.net).

Now, I maybe can guess what you might be thinking.
> _Another SDK? Just for glorified cURL?_

Maybe the best part of .NET is the developer experience. It's on your favorite package manager, there aren't any ".NET version managers" - you just install the version you want and it works, and you install _tools_ globally, not _packages_, so you couldn't even break your installation just by installing packages if you tried.

### Polyglot Notebooks
The .NET SDK already comes with all the F# tools we need to run these commands in the terminal with `fsi`, but this experience works much better with Polyglot Notebooks over VSCode with `code --install-extension ms-dotnettools.dotnet-interactive-vscode`.

## Syntax Sugar 🍬
The first package I'd like to show is:

In [1]:
#r "nuget: FsHttp"

open FsHttp

Installed Packages FsHttp, 14.5.1

Postman is useful because we - as humans - don't always remember the exact names of all the headers of the HTTP specification. But we kinda do at least know what a HTTP request is supposed to look like.

FsHttp uses an F# feature called computation expressions (see builder pattern, monad) to wrap `System.Net.Http` so _it actually looks like an HTTP request_.

In [2]:
http {
    GET "https://en.wiktionary.org/w/api.php?action=query&titles=%E5%AE%9D&prop=revisions&rvprop=content&rvsection=9&format=json"
}
|> Request.send
|> Response.toFormattedText

{
  "batchcomplete": "",
  "warnings": {
    "main": {
      "*": "Subscribe to the mediawiki-api-announce mailing list at \u003Chttps://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/\u003E for notice of API deprecations and breaking changes. Use [[Special:ApiFeatureUsage]] to see usage of deprecated features by your application."
    },
    "revisions": {
      "*": "Because \u0022rvslots\u0022 was not specified, a legacy format has been used for the output. This format is deprecated, and in the future the new format will always be used."
    }
  },
  "query": {
    "pages": {
      "14156": {
        "pageid": 14156,
        "ns": 0,
        "title": "\u5B9D",
        "revisions": [
          {
            "contentformat": "text/x-wiki",
            "contentmodel": "wikitext",
            "*": "==Japanese==\n{{ja-kanji forms|\u5B9D|\u5BF6}}\n\n===Kanji===\n{{ja-kanji|grade=6|rs=\u5B8005|kyu=\u5BF6}}\n\n# [[precious]] [[objects]]\n# [[worldly]] [[goods

Postman's UI makes it ease to identify the headers you need, but Postman is less trivial to edit and document with (see Postman collections).

FsHttp requests stored in fsi/ipynb files are just that - files, so they're dead simple to edit and document, but IntelliSense makes it trivial to write the exact request you want. Fun!

![bang](media/fshttp-intellisense0.gif)

[HTTP files](https://learn.microsoft.com/en-us/aspnet/core/test/http-files) can be a great alternative to Postman. Simply put - HTTP files store HTTP requests in text form, and your editor tooling can help you fill out the appropriate request and execute it.

![http file UI in Visual Studio](media/http-file-use0.png)

You can even use HTTP files in the form of HttpRepl blocks in a polyglot notebook!

In [3]:
Start-ThreadJob -ScriptBlock {
    # Define the URL and path
    $url = "http://localhost:8080/foo"

    # Define the content to write
    $content = "So a foo walks into a bar..."

    # Create an HttpListener
    $listener = [System.Net.HttpListener]::new()
    $listener.Prefixes.Add("http://localhost:8080/")

    # Start the listener
    $listener.Start()
    Write-Host "Listening on http://localhost:8080/"

    $context = $listener.GetContext()
    $response = $context.Response

    $buffer = [System.Text.Encoding]::UTF8.GetBytes($content)
    $response.ContentLength64 = $buffer.Length
    $response.OutputStream.Write($buffer, 0, $buffer.Length)
    $response.OutputStream.Close()
}
| Out-Null

In [4]:
GET http://localhost:8080

Name,Value
traceparent,00-d014d1bb9493ce82c59e4937d559bf7d-f62abc9ef7a72494-00
Name,Value
Server,Microsoft-HTTPAPI/2.0
Date,"Wed, 04 Sep 2024 02:37:34 GMT"
Content-Length,28


In [5]:
Stop-Job -Id 1
Remove-Job -Id 1

So far, we haven't found a strong reason to use FsHttp over HTTP files, but there's one big advantage FsHttp has over HTTP files: __programmability__.

## Data as First-Class Citizens 🗳️

If you're familiar with Python or JavaScript, you might be used to dot-navigating your way through API responses but having to do data validation manually. If you've used C#, you're probably used to getting a lot of data validation for cheap with `JsonDeserialize`, while still having to define your schema up front with `record`s and `class`es.

With F# type providers, you can actually get the benefit of both of these with neither of the costs. Let me show you what I mean:

In [6]:
#r "nuget: FSharp.Data"
open FSharp.Data

Installed Packages FSharp.Data, 6.4.0

In [7]:
type GitHubRepositories = JsonProvider<
"""
{
  "items": [
    {
      "id": 29048891,
      "name": "fsharp",
      "full_name": "dotnet/fsharp"
    }
  ]
}
""">

GitHubRepositories.Load("https://api.github.com/search/repositories?q=language:fsharp&per_page=5")
  .Items
  |> Array.map (_.FullName)

[ dotnet/fsharp, fable-compiler/Fable, fsharp/fsharp, giraffe-fsharp/Giraffe, fsprojects/Paket ]

Above, we gave `JsonProvider` an inline sample of our schema. This sample got ingested by the build process and was used to generate the rest of the `GitHubRepositories` type.

Passing in hard-coded or file referenced samples like this is recommended in real applications since the data becomes part of the build process. However, for just experimenting in an interactive session, it's perfectly fine to directly pass an API endpoint to `JsonProvider`.

In [8]:
type GitHubRepositories = JsonProvider<"https://api.github.com/search/repositories?q=language:fsharp&per_page=5">

GitHubRepositories.GetSample()
    .Items
    |> Array.map (fun x -> x.FullName, x.StargazersCount)

index value 0 (dotnet/fsharp, 3862) Item1 dotnet/fsharp Item2 3862 1 (fable-compiler/Fable, 2887) Item1 fable-compiler/Fable Item2 2887 2 (fsharp/fsharp, 2172) Item1 fsharp/fsharp Item2 2172 3 (giraffe-fsharp/Giraffe, 2106) Item1 giraffe-fsharp/Giraffe Item2 2106 4 (fsprojects/Paket, 2014) Item1 fsprojects/Paket Item2 2014

Because FsHttp is a wrapper for building *requests* and FSharp.Data is a library for intepreting (well, data generally, but in our case) *responses*, and because .NET has an awesome "work together" culture instead of "rebuild a worse version yourself 'from ground up'" culture, these two libraries work together rather flawlessly ✨:

In [9]:
type Kanji = JsonProvider<"https://kanjiapi.dev/v1/kanji/力">

let stream =
    http {
        GET "https://kanjiapi.dev/v1/kanji/生"
    }
    |> Request.send
    |> Response.toStream

let 生 = Kanji.Load(stream)

Array.concat (seq { 生.KunReadings; 生.OnReadings; })

[ -う, い.かす, い.きる, い.ける, う.まれ, う.まれる, う.む, うま.れる, うまれ, お.う, き, な.す, な.る, なま, なま-, は.える, は.やす, む.す, ショウ, セイ ]

## Next Steps 🤔
Don't wait! If a lot of things in this blog post seemed unfamiliar, you will assume they're hard and have already created a mental blockade. You will need an HTTP client later, but the energy to set up FsHttp will seem greater than dealing with the overhead of your Least Common Denominator tool, but over time that overhead will pile up and contribute to developer burn out.

Refresh your toolkit today! Start installing [dot.net](https://dot.net), take a break from the computer screen and stretch those legs, then [![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://github.dev/johnW-ret/fstandsforfun) in the browser or [![Open in VS Code](media/open-in-vscode.svg)](vscode://vscode.git/clone?url=https://github.com/johnW-ret/fstandsforfun)!

Happy coding!
